In [1]:
import numpy as np

# ---------- Data ----------
text = "I Love AI"
chars = sorted(list(set(text)))
vocab = {ch:i for i,ch in enumerate(chars)}
ivocab = {i:ch for ch,i in vocab.items()}
data = [vocab[ch] for ch in text]
V = len(chars)

# ---------- Small RNN params ----------
H = 8              # hidden size (small)
lr = 0.5
Wxh = np.random.randn(H, V) * 0.01   # (H, V)
Whh = np.random.randn(H, H) * 0.01   # (H, H)
Why = np.random.randn(V, H) * 0.01   # (V, H)
bh = np.zeros((H,1))
by = np.zeros((V,1))

def softmax(x):
    e = np.exp(x - np.max(x))
    return e / e.sum(axis=0)

# ---------- Training (short) ----------
epochs = 2000
for ep in range(epochs):
    h_prev = np.zeros((H,1))
    total_loss = 0.0
    for t in range(len(data)-1):
        # one-hot input and target
        x = np.zeros((V,1)); x[data[t]] = 1
        target = data[t+1]

        # forward
        h = np.tanh(Wxh.dot(x) + Whh.dot(h_prev) + bh)   # (H,1)
        y = Why.dot(h) + by                              # (V,1)
        p = softmax(y)

        # loss
        loss = -np.log(p[target,0] + 1e-9)
        total_loss += loss

        # gradients (one-step)
        dy = p.copy(); dy[target,0] -= 1                 # (V,1)
        dWhy = dy.dot(h.T)                               # (V,H)
        dby  = dy
        dh = Why.T.dot(dy) * (1 - h*h)                   # (H,1)
        dWxh = dh.dot(x.T)                               # (H,V)
        dWhh = dh.dot(h_prev.T)                          # (H,H)
        dbh  = dh

        # update
        Wxh -= lr * dWxh
        Whh -= lr * dWhh
        Why -= lr * dWhy
        bh  -= lr * dbh
        by  -= lr * dby

        h_prev = h

    if ep % 500 == 0:
        print(f"Epoch {ep}, loss={total_loss:.4f}")

print("Training done")

# ---------- Simple generation ----------
def generate(seed, length=30):
    h = np.zeros((H,1))
    ch = seed[0]
    out = seed
    for _ in range(length):
        x = np.zeros((V,1)); x[vocab[ch]] = 1
        h = np.tanh(Wxh.dot(x) + Whh.dot(h) + bh)
        y = Why.dot(h) + by
        p = softmax(y).ravel()
        idx = np.random.choice(range(V), p=p)
        ch = ivocab[idx]
        out += ch
    return out

print("Generated:", generate("I", 30))


Epoch 0, loss=17.1117
Epoch 500, loss=0.0045
Epoch 1000, loss=0.0023
Epoch 1500, loss=0.0016
Training done
Generated: I Love AIe AIe AIe AIe AIvLAve 
